In [1]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import time
import sqlite3 as sql

In [2]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20230405_083949'

In [3]:
### Importer un module de fonctions crée ad hoc
## ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans le même dossier que le présent carnet Jupyter afin que l'importation
#   fonctionne correctement
import sparql_functions as spqf

In [ ]:
### Recharger le module après modification des fonctions dans le fichier du module
# désactivé #  reload(spqf)

## Queries

In [ ]:
### Requête générique qui compte les statements par propriété

qp = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX ontome: <https://ontome.net/>
PREFIX geov: <http://geovistory.org/resource/>

SELECT ?predicate ?label (count(*) as ?eff)
WHERE {
  ?subject ?predicate ?object .
  ?predicate rdfs:label ?label 
}
group by ?predicate ?label
order by desc(?eff)
"""

In [ ]:
### Requête générique qui compte les instances de chaque classe

qc = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX ontome: <https://ontome.net/>
PREFIX geov: <http://geovistory.org/resource/>

SELECT ?class ?label (count(*) as ?eff)
WHERE {
  ?subject a ?class .
  ?class rdfs:label ?label 
}
group by ?class ?label
order by desc(?eff)
"""

In [37]:
### URI de DBPedia

q1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX ontome: <https://ontome.net/ontology/>
PREFIX geov: <http://geovistory.org/resource/>

## get persons and their URIs with birthyear
SELECT ?subject ?entityLabel (STR(?birthYear) as ?strYear) ?dbUri

WHERE {
  ?subject a ontome:c21;
       rdfs:label ?entityLabel;
      ontome:p1943 / ontome:p1843 ?strUri;
     ^ontome:p86 / ontome:p4 / ontome:p72 / time:year ?birthYear.
 BIND(URI(?strUri) as ?dbUri)
}
LIMIT 100
"""

In [23]:
### Ajouter le lieu de naissance de DBPedia

q2 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX ontome: <https://ontome.net/ontology/>
PREFIX geov: <http://geovistory.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?subject ?entityLabel (STR(?birthYear) as ?strYear) ?dbUri ?birthPlace

WHERE {
  ?subject a ontome:c21;
       rdfs:label ?entityLabel;
      ontome:p1943 / ontome:p1843 ?strUri;
     ^ontome:p86 / ontome:p4 / ontome:p72 / time:year ?birthYear.
 BIND(URI(?strUri) as ?dbUri)
  
    
  SERVICE <https://dbpedia.org/sparql/> {
  ?dbUri dbo:birthPlace ?birthPlace
  }
  
}
ORDER BY ?subject
# LIMIT 100
"""

In [71]:
### Ajouter le type de lieu de naissance de DBPedia

q3 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX ontome: <https://ontome.net/ontology/>
PREFIX geov: <http://geovistory.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?subject ?entityLabel (STR(?birthYear) as ?strYear) ?dbUri ?birthPlace ?placeType

WHERE {
  ?subject a ontome:c21;
       rdfs:label ?entityLabel;
      ontome:p1943 / ontome:p1843 ?strUri;
     ^ontome:p86 / ontome:p4 / ontome:p72 / time:year ?birthYear.
 BIND(URI(?strUri) as ?dbUri)
  
    
  SERVICE <https://dbpedia.org/sparql/> {
  ?dbUri dbo:birthPlace ?birthPlace .
  ?birthPlace a ?placeType .
  }
  
}
ORDER BY ?subject
# LIMIT 100
"""

In [48]:
### Ajouter le type de lieu de naissance de DBPedia

q4 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX ontome: <https://ontome.net/ontology/>
PREFIX geov: <http://geovistory.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?subject ?entityLabel (STR(?birthYear) as ?strYear) ?dbUri ?abstract
WHERE {
  ?subject a ontome:c21;
       rdfs:label ?entityLabel;
      ontome:p1943 / ontome:p1843 ?strUri;
     ^ontome:p86 / ontome:p4 / ontome:p72 / time:year ?birthYear.
 BIND(URI(?strUri) as ?dbUri)
  
    
  SERVICE <https://dbpedia.org/sparql/> {
    ?dbUri dbo:abstract ?abstract.
    ### https://sodocumentation.net/sparql/topic/5210/working-with-language-tags
    FILTER langMatches(lang(?abstract), 'en')
    BIND(lang(?abstract) as ?abstractLang)
  }
  
}
ORDER BY ?subject
# LIMIT 100
"""

## Query execution

*  https://sparql.geovistory.org/api_v1_project_153

In [5]:
addr = "https://sparql.geovistory.org/api_v1_project_153"

In [67]:
### Exécuter la requête SPARQL enveloppée par cette fonction, 
# elle se trouve dans le biliothèque-utilisateur _sparql_functions.py_
# le premier paramètre correspond au point d'accès SPARQL, le deuxième à la requête
qr = spqf.get_json_sparql_result(addr,q3)

<class 'dict'>


In [68]:
# Nombre de lignes de la réponse
len(qr['results']['bindings'])

47447

In [72]:
# Inspecter les n premières lignes du résultat

def inspect_n_lines (qr, n = 1):
    i = 0
    for l in qr['results']['bindings']:
        if i < n:
            print(l)
            i += 1

In [ ]:
inspect_n_lines (qr, n = 2)

In [74]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr) ]
# r

In [75]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:2]

47447


[['http://geovistory.org/resource/i25894',
  'Félix Dupanloup',
  '1802',
  'http://dbpedia.org/resource/Félix_Dupanloup',
  'http://dbpedia.org/resource/Haute-Savoie',
  'http://dbpedia.org/class/yago/AdministrativeDistrict108491826'],
 ['http://geovistory.org/resource/i25894',
  'Félix Dupanloup',
  '1802',
  'http://dbpedia.org/resource/Félix_Dupanloup',
  'http://dbpedia.org/resource/Haute-Savoie',
  'http://dbpedia.org/class/yago/City108524735']]

## Requêtes d'insertion de données dans une base de données SQLite 

La finalité est d'avoir les données sous la main pour exploration ultérieure

La base a été remplie en été 2022

In [56]:
db_addr = "proj_153_dbpedia_and_co.sqlite"

### Personnes avec notices

In [58]:
proj_153_persons = pd.DataFrame(r) ; proj_153_persons.shape

(1455, 5)

In [59]:
proj_153_persons.columns = ['gv_uri', 'name', 'birthYear', 'dbpedia_uri', 'dbpedia_abstract']

In [60]:
proj_153_persons.head(3)

,gv_uri,name,birthYear,dbpedia_uri,dbpedia_abstract
0,http://geovistory.org/resource/i25892,Quirino Gasparini,1721,http://dbpedia.org/resource/Quirino_Gasparini,Quirino Gasparini (24 October 1721 – 26 Septem...
1,http://geovistory.org/resource/i25893,Giacomo Vaghi,1901,http://dbpedia.org/resource/Giacomo_Vaghi,Giacomo Vaghi (21 November 1901 - 29 April 197...
2,http://geovistory.org/resource/i25894,Félix Dupanloup,1802,http://dbpedia.org/resource/Félix_Dupanloup,Mgr. Félix Antoine Philibert Dupanloup (3 Janu...


### Personnes avec lieux de naissance

In [77]:
proj_153_persons_birthPlaces = pd.DataFrame(r) ; proj_153_persons_birthPlaces.shape

(47447, 6)

In [82]:
proj_153_persons_birthPlaces.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47447 entries, 0 to 47446
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       47447 non-null  object
 1   1       47447 non-null  object
 2   2       47447 non-null  object
 3   3       47447 non-null  object
 4   4       47447 non-null  object
 5   5       47447 non-null  object
dtypes: object(6)
memory usage: 2.2+ MB


In [83]:
proj_153_persons_birthPlaces.head(2)

,0,1,2,3,4,5
0,http://geovistory.org/resource/i25894,Félix Dupanloup,1802,http://dbpedia.org/resource/Félix_Dupanloup,http://dbpedia.org/resource/Haute-Savoie,http://dbpedia.org/class/yago/AdministrativeDi...
1,http://geovistory.org/resource/i25894,Félix Dupanloup,1802,http://dbpedia.org/resource/Félix_Dupanloup,http://dbpedia.org/resource/Haute-Savoie,http://dbpedia.org/class/yago/City108524735


In [84]:
proj_153_persons_birthPlaces.columns = ['gv_uri', 'name', 'birthYear', 'dbpedia_uri', 'dbpedia_birthPlace', 'dbpedia_birthPlaceType']

In [85]:
proj_153_persons_birthPlaces.head(3)

,gv_uri,name,birthYear,dbpedia_uri,dbpedia_birthPlace,dbpedia_birthPlaceType
0,http://geovistory.org/resource/i25894,Félix Dupanloup,1802,http://dbpedia.org/resource/Félix_Dupanloup,http://dbpedia.org/resource/Haute-Savoie,http://dbpedia.org/class/yago/AdministrativeDi...
1,http://geovistory.org/resource/i25894,Félix Dupanloup,1802,http://dbpedia.org/resource/Félix_Dupanloup,http://dbpedia.org/resource/Haute-Savoie,http://dbpedia.org/class/yago/City108524735
2,http://geovistory.org/resource/i25894,Félix Dupanloup,1802,http://dbpedia.org/resource/Félix_Dupanloup,http://dbpedia.org/resource/Haute-Savoie,http://dbpedia.org/class/yago/District108552138


In [87]:
conn = sql.connect(db_addr)

In [65]:
### Décommenter si nécessaire
## proj_153_persons.to_sql('proj_153_persons', conn, if_exists='replace', index=False)

In [88]:
### Décommenter si nécessaire
## proj_153_persons_birthPlaces.to_sql('proj_153_persons_birthPlaces', conn, if_exists='replace', index=False)

47447

In [89]:
conn.close()

In [91]:
cn = sql.connect(db_addr)

In [92]:
### Décommenter si nécessaire
q = """
WITH tw1 AS (
SELECT DISTINCT dbpedia_uri, dbpedia_birthPlace 
FROM proj_153_persons_birthPlaces
)
SELECT dbpedia_uri, COUNT(*) AS eff, group_concat(dbpedia_birthPlace, ' | ')
FROM tw1 
GROUP BY dbpedia_uri 
HAVING COUNT(*) > 1
ORDER BY eff DESC; 
"""

In [93]:
### Création du conteneur du résultat de la requête
cur = cn.cursor()
cur

In [95]:
### Exécuter la requête et récupérer le résultat
cur.execute(q)
# La méthode 'fetchall' permet de récupérer toutes les lignes
r = cur.fetchall()

In [101]:
conn.close()

In [102]:
### Inspecter le résultat:
# longueur, type du résultat et premières lignes du résultat
# On observe qu'il y a des trous et des répétitions:
# plusieurs personnes on la même année de naissance
print(len(r), '\n-----\n', r[:5])

544 
-----
 [('http://dbpedia.org/resource/Rachel_Félix', 4, 'http://dbpedia.org/resource/Aargau | http://dbpedia.org/resource/Mumpf | http://dbpedia.org/resource/Rheinfelden_(Aargau) | http://dbpedia.org/resource/Switzerland'), ('http://dbpedia.org/resource/Léon_Bel', 4, 'http://dbpedia.org/resource/France | http://dbpedia.org/resource/Franche-Comté | http://dbpedia.org/resource/Jura_(department) | http://dbpedia.org/resource/Orgelet'), ('http://dbpedia.org/resource/Jules_Bel', 4, 'http://dbpedia.org/resource/Chambéria | http://dbpedia.org/resource/France | http://dbpedia.org/resource/Franche-Comté | http://dbpedia.org/resource/Jura_(department)'), ('http://dbpedia.org/resource/Gustave_Colin', 4, 'http://dbpedia.org/resource/Doubs | http://dbpedia.org/resource/France | http://dbpedia.org/resource/Franche-Comté | http://dbpedia.org/resource/Pontarlier'), ('http://dbpedia.org/resource/Gertrud_Wolle', 4, 'http://dbpedia.org/resource/Alsace | http://dbpedia.org/resource/Arrondissement_of_

In [103]:
[e[1:] for e in r[:5]]

[(4,
  'http://dbpedia.org/resource/Aargau | http://dbpedia.org/resource/Mumpf | http://dbpedia.org/resource/Rheinfelden_(Aargau) | http://dbpedia.org/resource/Switzerland'),
 (4,
  'http://dbpedia.org/resource/France | http://dbpedia.org/resource/Franche-Comté | http://dbpedia.org/resource/Jura_(department) | http://dbpedia.org/resource/Orgelet'),
 (4,
  'http://dbpedia.org/resource/Chambéria | http://dbpedia.org/resource/France | http://dbpedia.org/resource/Franche-Comté | http://dbpedia.org/resource/Jura_(department)'),
 (4,
  'http://dbpedia.org/resource/Doubs | http://dbpedia.org/resource/France | http://dbpedia.org/resource/Franche-Comté | http://dbpedia.org/resource/Pontarlier'),
 (4,
  'http://dbpedia.org/resource/Alsace | http://dbpedia.org/resource/Arrondissement_of_Thann | http://dbpedia.org/resource/Germany | http://dbpedia.org/resource/Urbès')]